COLETA DOS DADOS


In [1]:
#Os imports não são finais
import pandas as pd
#import sqlite3
import numpy as np
import requests

#Para gerar o arquivo .csv sem tratamento a partir da página .html
dataHTML = pd.read_html('http://www.nuforc.org/webreports/ndxevent.html')
dataHTML[0].to_csv('file.csv')
dataCSV = pd.read_csv('file.csv')

#Para fazer o tratamento do DataFrame do pandas:
keep_col = ['Reports','Count']  #para o tratamento do arquivo com os index para o .csv
newData = dataCSV[keep_col]

#E agora para deixar apenas os dados do intervalo de tempo desejado
newData = newData.iloc[45:285]

#Gerar o novo arquivo usado como o dos dados que queremos acessar
newData.to_csv("OVNIs_Index.csv", index = False)
indexOVNIs = pd.read_csv('OVNIs_Index.csv')

#A partir daqui fazemos o tratamento para inserir o conteúdo das tabelas HTML dentro de cada link
#Método para acessar um link com uma data específica
def tableHMTL(date):
    url = 'http://www.nuforc.org/webreports/ndxe' + date + '.html'
    page = requests.get(url)
    dfHMTL = pd.read_html(page.text)
    dfHMTL = pd.DataFrame(np.concatenate(dfHMTL), columns=columns)
    return dfHMTL

#Passar dataOVNIs['Reports'] como index para chamar links do dataHTML
columns = ['Data/Hora','Cidade','Estado','Forma','Duracao','Descricao','Postagem']
tablesOVNIS = []
tablesOVNIS = pd.DataFrame(tablesOVNIS, columns=columns)

#Esse bloco acessa cada link da página de relatos de OVNIS e imprime cada relato
#passando dataOVNIs['Reports'] como index para chamar links do dataHTML
for i in indexOVNIs['Reports']:
    data_relato = str(i)
    data_split = data_relato.split("/")
    data_split.reverse()
    date = ''.join(data_split)
    tablesOVNIS = pd.concat([tableHMTL(date),tablesOVNIS])

tablesOVNIS.to_csv('OVNIS.csv')
tablesOVNIS

,Data/Hora,Cidade,Estado,Forma,Duracao,Descricao,Postagem
0,11/30/97 22:15,Cardinal (Canada),ON,Cone,7 min's,My wife and I were just on our way to bed when...,12/2/00
1,11/30/97 18:00,Sacramento,CA,Flare,30 seconds,I think we saw something simular as discribed ...,6/2/98
2,11/30/97 02:00,LeMars (IA)/Denver (CO) (uncertain; traveling ...,NE,Light,approx. 5 minutes,"Multi-colored ""pole,"" appearing to hang in the...",12/2/00
3,11/29/97 20:00,Gorman (60 miles north of Los Angeles),CA,Light,10-min.,I was coyote calling on top of a mountain at n...,6/2/98
4,11/29/97 18:15,Orcas Island,WA,Light,1 hour,a bright light in the ssw sky slowly moves dow...,6/2/98
...,...,...,...,...,...,...,...
490,10/1/17 11:25,Denver,CO,Triangle,15 minutes,A cluster of whitish balls with blue chevron w...,10/5/17
491,10/1/17 10:00,Mount pleasant,SC,Light,Still going,White light in the day sky,10/5/17
492,10/1/17 08:10,Mechanicsburg,PA,Triangle,10 minutes,3 bright orange lights in a triangular shape j...,10/5/17
493,10/1/17 04:30,DeLand,FL,Light,20 minutes,I noticed whay appeared to be one very bright ...,10/5/17


Exploração com PandasSQL


LISTA DE INTENS OBRIGATÓRIOS PARA A EXPLORAÇÃO

In [58]:
# 1- Saber a quantidade de linhas, observações ou variáveis que foram coletadas. -> NÚMERO DE LINHAS 
import pandas as pd

print('A quantidade de linhas e colunas coletadas são:', tablesOVNIS.shape)

A quantidade de linhas e colunas coletadas são: (100964, 7)


In [59]:
# 2- Quantos relatos ocorreram por estado em ordem decrescente?

q='''
  SELECT Estado, COUNT(*)
  FROM tablesOVNIS
  GROUP BY Estado
  HAVING COUNT(*)
  ORDER BY COUNT(*) desc
'''

consulta = pandasql.sqldf(q, locals())
consulta

,Estado,COUNT(*)
0,CA,11527
1,None,7247
2,FL,5655
3,WA,4956
4,TX,4172
...,...,...
63,NT,21
64,YT,21
65,PE,20
66,PR,18


In [69]:
# 3- Remover possíveis campos vazios (sem estado)
tablesOVNIS.drop(tablesOVNIS.index[tablesOVNIS['Estado'] == 'None'], inplace = True)
tablesOVNIS.drop(tablesOVNIS.index[tablesOVNIS['Estado'] == None], inplace = True)

# Remover possíveis campos NaN
tablesOVNIS['Estado'].dropna()

tablesOVNIS

,Data/Hora,Cidade,Estado,Forma,Duracao,Descricao,Postagem
0,11/30/97 22:15,Cardinal (Canada),ON,Cone,7 min's,My wife and I were just on our way to bed when...,12/2/00
1,11/30/97 18:00,Sacramento,CA,Flare,30 seconds,I think we saw something simular as discribed ...,6/2/98
2,11/30/97 02:00,LeMars (IA)/Denver (CO) (uncertain; traveling ...,NE,Light,approx. 5 minutes,"Multi-colored ""pole,"" appearing to hang in the...",12/2/00
3,11/29/97 20:00,Gorman (60 miles north of Los Angeles),CA,Light,10-min.,I was coyote calling on top of a mountain at n...,6/2/98
4,11/29/97 18:15,Orcas Island,WA,Light,1 hour,a bright light in the ssw sky slowly moves dow...,6/2/98
...,...,...,...,...,...,...,...
490,10/1/17 11:25,Denver,CO,Triangle,15 minutes,A cluster of whitish balls with blue chevron w...,10/5/17
491,10/1/17 10:00,Mount pleasant,SC,Light,Still going,White light in the day sky,10/5/17
492,10/1/17 08:10,Mechanicsburg,PA,Triangle,10 minutes,3 bright orange lights in a triangular shape j...,10/5/17
493,10/1/17 04:30,DeLand,FL,Light,20 minutes,I noticed whay appeared to be one very bright ...,10/5/17


In [65]:
# 4- Limitar a análise aos estados dos Estados Unidos.

import pandas
!pip install -U pandasql
import pandasql

def filtro_EUA(filename):

  # Tabela de Estados dos EUA
  ovnis_data = pandas.read_csv(filename)
  
  q='''
      SELECT *
      FROM ovnis_data
      WHERE Estado in ('AK','AL','AR','AZ','CA','CO','CT','DE','FL','GA','HI','IA','ID','IL',
      'IN','KS','KY','LA','MA','MD','ME','MI','MN','MO','MS','MT','NC','ND',
      'NE','NH','NJ','NM','NV','NY','OH','OK','OR','PA','RI','SC','SD','TN',
      'TX','UT','VT','VA','WA','WI','WV','WY')
      ORDER BY Estado
  '''
  filtro_est_USA = pandasql.sqldf(q, locals())

  OVNIS_limpo = pd.DataFrame(filtro_est_USA)
  OVNIS_limpo.to_csv("OVNIs_limpo.csv", index = False)
  
  return OVNIS_limpo

filtro_EUA('./OVNIS.csv')


,Unnamed: 0,Data/Hora,Cidade,Estado,Forma,Duracao,Descricao,Postagem
0,37,11/17/97 03:00,Wasilla,AK,Triangle,3 minutes,A huge trianglar object moving in a true north...,9/2/05
1,37,1/15/98 13:00,Alaska (remote),AK,Disk,20 seconds,saucer shaped object passed directly above me,3/16/00
2,58,1/8/98 22:38,Fairbanks,AK,Oval,3 Secs,A large phlorescent green oval shaped object m...,2/16/99
3,55,3/15/98 20:30,Anchorage,AK,Oval,15 minutes,"Driving home after daughters birthday, noticed...",6/18/98
4,57,3/15/98 20:00,North Pole,AK,Sphere,90 sec,"Red orb, blue orb...similar to the Marfa Lights",5/24/05
...,...,...,...,...,...,...,...,...
89271,224,7/17/17 21:00,Cody,WY,Cylinder,30 seconds,Craft heading Northeast at a high rate of spee...,7/23/17
89272,293,7/13/17 04:00,Pinedale,WY,Sphere,None,Star-like orb flashing colors in night sky.,7/23/17
89273,34,10/28/17 20:00,Cody and Wapiti (between),WY,Fireball,10 seconds,There were fireballs coming down. ((anonymous ...,11/3/17
89274,282,10/15/17 20:15,Douglas,WY,Sphere,3 minutes,2 of my friends and I were standing on my balc...,10/19/17


In [70]:
# 5- Consulta por cidades, com o objetivo de saber quais contêm o maior número 
  # de relatos (cidades que apresentem ao menos 10 relatos).

df2 = pd.read_csv('./OVNIs_limpo.csv')

q='''
  SELECT Cidade, COUNT(*)
  FROM df2
  GROUP BY Cidade
  HAVING COUNT(*) > 9
  ORDER BY COUNT(*) desc
'''

consulta1 = pandasql.sqldf(q, locals())
consulta1

,Cidade,COUNT(*)
0,Phoenix,562
1,Seattle,554
2,Portland,484
3,Las Vegas,477
4,San Diego,402
...,...,...
1862,Willow Springs,10
1863,Wolf Point,10
1864,Woodville,10
1865,Yarmouth,10


In [ ]:
# 6- Com o dado anterior, responder a seguinte pergunta: por que será que essa é 
  #a cidade que possui mais relatos?

'''
  Phoenix é a cidade que possui mais relatos devido a um evento denominado 
  "Luzes de Phoenix" que ocorreu no estado do Arizona e Nevada (EUA) e em Sonora (MEX)
  no ano de 1997. Os fenômenos ópticos ocorridos nessa região foram visto por milhares de pessoas,
  o que ocasionou uma série de registros de avistamento de OVNIS's no BD do site do NUFORC
'''

In [76]:
# 7- Fazer uma query exclusiva para o estado com maior número de relatos, buscando 
# cidades que possuam um número superior a 10 relatos. 

# Enfatizar a cidade, a quantidade de relatos e formato do objeto não identificado.

df3 = pd.read_csv('./OVNIs_limpo.csv')

q='''
  SELECT Estado, Cidade, COUNT(*), Forma
  FROM df3
  WHERE Estado = 'CA'
  GROUP BY Cidade
  HAVING COUNT(*) > 9
  ORDER BY COUNT(*) desc
'''

consulta2 = pandasql.sqldf(q, locals())
consulta2

,Estado,Cidade,COUNT(*),Forma
0,CA,San Diego,399,Other
1,CA,Los Angeles,385,Other
2,CA,Sacramento,243,Unknown
3,CA,San Jose,225,Oval
4,CA,San Francisco,199,Triangle
...,...,...,...,...
277,CA,Seal Beach,10,Flash
278,CA,South Pasadena,10,Circle
279,CA,Union City,10,Light
280,CA,Willits,10,Other
